In [1]:
import torch
from utils.attn_utils import *
from diffusers import ControlNetModel
from pipeline_decompose_and_realign_v2 import *
from diffusers import DPMSolverMultistepScheduler, EulerDiscreteScheduler, EulerAncestralDiscreteScheduler

device = torch.device("cuda")
controlnet_dict = {
    'depth': ControlNetModel.from_pretrained('/home/wangluozhou/pretrained_models/sd-controlnet-depth'),
    'normal': ControlNetModel.from_pretrained('/home/wangluozhou/pretrained_models/sd-controlnet-normal'),
    'canny': ControlNetModel.from_pretrained('/home/wangluozhou/pretrained_models/sd-controlnet-canny'),
    'pose': ControlNetModel.from_pretrained('/home/wangluozhou/pretrained_models/sd-controlnet-pose'),
}

pipe = DecomposeAndRealignPipeline.from_pretrained('/home/wangluozhou/pretrained_models/gligen-1-4-generation-text-box').to(device)
pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)
pipe.set_controlnet(controlnet_dict)

dataset_path = '/home/wangluozhou/projects/Decompose-and-Realign/data/metadata_complex.jsonl'
dataset = load_jsonl(dataset_path)
base_dir = os.path.dirname(dataset_path)
for data in dataset:
    for key, value in data.items():
        if key!='text' and key!='bbox':
            value['control_info'] = Image.open(os.path.join(base_dir, value['control_info'],f'{key}.png')).convert('RGB')

/home/wangluozhou/miniconda3/envs/llava/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-28 16:41:57.933103: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-28 16:41:57.986813: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-28 16:41:59.281223: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: 

[2023-11-28 16:42:03,188] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Loading pipeline components...: 100%|██████████| 5/5 [00:04<00:00,  1.06it/s]


In [2]:
data = dataset[0]
data['text']['control_info'] = 10

guidance_scale_dict = {
    'text':7,
    'bbox':4,
    'pose':5,
    'depth':2,
    'normal':3,
}
for key in data:
    data[key]['cfg'] = guidance_scale_dict[key]



data = sort_inputs(data)

In [ ]:
data

In [3]:
for i in range(0,100):
    output = pipe(
        inputs=data,
        negative_prompt='unnatural colors, bad proportions, worst quality',
        dr_scheduled_sampling_beta=0.5,
        height=512,
        width=512,
        generator=torch.Generator(device="cuda").manual_seed(i),
    )
    output.images[0].save(f'outputs/{str(i).zfill(3)}.png')

/home/wangluozhou/projects/Decompose-and-Realign/pipeline_decompose_and_realign_v2.py:570: FutureWarning: Accessing config attribute `cross_attention_dim` directly via 'UNet2DConditionModel' object attribute is deprecated. Please access 'cross_attention_dim' over 'UNet2DConditionModel's config object instead, e.g. 'unet.config.cross_attention_dim'.
  max_objs, self.unet.cross_attention_dim, device=device, dtype=self.text_encoder.dtype
/home/wangluozhou/projects/Decompose-and-Realign/pipeline_decompose_and_realign_v2.py:733: FutureWarning: Accessing config attribute `in_channels` directly via 'UNet2DConditionModel' object attribute is deprecated. Please access 'in_channels' over 'UNet2DConditionModel's config object instead, e.g. 'unet.config.in_channels'.
  num_channels_latents = self.unet.in_channels
100%|██████████| 50/50 [00:25<00:00,  1.93it/s]


In [ ]:
output = pipe(
    inputs=data,
    negative_prompt='unnatural colors, bad proportions, worst quality',
    dr_scheduled_sampling_beta=0.5,
    height=512,
    width=512,
    generator=torch.Generator(device="cuda").manual_seed(7),
)
output.images[0]